OBJECT ORIENTED PROGRAMMING 


In [9]:
#1) Class: Template/Type of objects. For example: String, Float, List, etc. We can also define our own class(es).
#2) Instance: = object. An actual instance of a class
x = 'abc'
type(x)

str

In [ ]:
#3) Method - what an object does. The class of an object would determine what an object can and cannot do
dir(x)

In [12]:
####EXAMPLE OF DEFINING A CLASS
class PartyAnimal:
    x = 0
    def party(self):
        self.x = self.x + 1
        print('So far', self.x)


In [20]:
#Using the class to create new object and use methods
an = PartyAnimal()
an.party()
an.party()
an.party()

So far 1
So far 2
So far 3


In [25]:
type(an)

__main__.PartyAnimal

In [ ]:
dir(an) #will contains the "party" method defined

In [30]:
#Use of constructor and destructor method -- optional 
class PartyAnimal:
    x = 0
    
    def __init__(self):             ######construction method - usually when defining variable
        print('I am constructed')
    
    def party(self):
        self.x = self.x + 1
        print('So far', self.x)
    
    def __del__(self):
        print('I am destructed', self.x)    ######less commonly used

In [31]:
an = PartyAnimal()
an.party()
an.party()
an = 42
print('New value of an is', an)

I am constructed
So far 1
So far 2
I am destructed 2
New value of an is 42


In [54]:
####Another example for defining class - has an input parameter in the beginning

In [49]:
class PartyAnimal:
    x = 0
    name = ""
    def __init__(self, input_name):             ######construction method - usually when defining variable
        self.name = input_name
        print(self.name, 'constructed')
    
    def party(self):
        self.x = self.x + 1
        print(self.name, 'party count', self.x)

In [50]:
hi = PartyAnimal("Sally")

Sally constructed


In [51]:
print(hi.name)

Sally


In [52]:
hi.party()
hi.party()
hi.party()

Sally party count 1
Sally party count 2
Sally party count 3


In [53]:
### Inheritance: Create a new class that inherit the capabilities of an existing class and add a little bit more
### The new class (child) has all the capabilities of the old class (parent) and some more

In [62]:
##creating a child class:
class FootballFan(PartyAnimal):
    points = 0
    def touchdown(self):
        self.points = self.points + 7
        self.party()
        print(self.name, 'points', self.points)

In [63]:
s = PartyAnimal('Kaka')
s.party()

j = FootballFan('Jim') 
j.party()                     #######the child class has the parent's capability
j.party()
j.touchdown()                 #######and the new capability

Kaka constructed
Kaka party count 1
Jim constructed
Jim party count 1
Jim party count 2
Jim party count 3
Jim points 7


USING SQLITE 

In [ ]:
import sqlite3

conn = sqlite3.connect('emaildb.sqlite')   ###connect to a database - will create new one if not exists
cur = conn.cursor()                        ###cursor - kind of a handle attaching to the current script/data etc.


###execute a sql
cur.execute('DROP TABLE IF EXISTS Counts') 
cur.execute('''CREATE TABLE Counts (email TEXT, count INTEGER)''')

###execute multiple sqls
cur.executescript('''
DROP TABLE IF EXISTS Artist;
DROP TABLE IF EXISTS Album;
DROP TABLE IF EXISTS Track;

CREATE TABLE Artist (
    id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    name    TEXT UNIQUE
);

CREATE TABLE Album (
    id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    artist_id  INTEGER,
    title   TEXT UNIQUE
);''') 

###select sql
sqlstr = 'SELECT email, count FROM Counts ORDER BY count DESC LIMIT 10'  ##select will attach result to cursor but not display
for row in cur.execute(sqlstr):                                          ##result is a set of tuples that we can loop through
    print(str(row[0]), row[1])                                           ##we will then have to print or fetch (see below)

###if the cursor is currently attachd to the result of a select sql - we can use fetch
cur.fetchone()                             ###fetch first row -- 1 tuple
cur.fetchall()                             ###fetch all row -- all tuples
cur.fetchmany(number_of_records)           ###fetch a number of rows -- list of tuples
    
###execute a sql with a parameter set in a tuple
###this is supposed to prevent sql injection (a type of hacking) - not sure I understand how yet
cur.execute('''INSERT INTO Counts (email, count)   
                VALUES (?, 1)''', (email,))        
cur.execute('UPDATE Counts SET count = count + 1 WHERE email = ?',
                    (email,))


###ending statements
conn.commit()                              ###commit the data to database
cur.close()                                ###close the cursor after done

In [ ]:
#########SOME SPECIFIC COMMAND IN SQLITE   #################
INSERT OR IGNORE INTO tablename (column1, column2) VALUES (value1, value2)
INSERT OR REPLACE INTO tablename (column1, column2) VALUES (value1, value2)



In [ ]:
#########EXAMPLE    ######################################

import sqlite3

conn = sqlite3.connect('emaildb.sqlite')
cur = conn.cursor()

cur.execute('DROP TABLE IF EXISTS Counts')

cur.execute('''
CREATE TABLE Counts (email TEXT, count INTEGER)''')

fname = input('Enter file name: ')
if (len(fname) < 1): fname = 'mbox.txt'
fh = open(fname)
for line in fh:
    if not line.startswith('From: '): continue
    pieces = line.split()
    email = pieces[1]
    cur.execute('SELECT count FROM Counts WHERE email = ? ', (email,))
    row = cur.fetchone()
    if row is None:
        cur.execute('''INSERT INTO Counts (email, count)
                VALUES (?, 1)''', (email,))
    else:
        cur.execute('UPDATE Counts SET count = count + 1 WHERE email = ?',
                    (email,))
    conn.commit()

# https://www.sqlite.org/lang_select.html
sqlstr = 'SELECT email, count FROM Counts ORDER BY count DESC LIMIT 10'

for row in cur.execute(sqlstr):
    print(str(row[0]), row[1])

cur.close()


Enter file name: 


In [21]:
import sqlite3

conn = sqlite3.connect('emaildb.sqlite')   ###connect to a database - will create new one if not exists
cur = conn.cursor()                        ###cursor - kind of a handle attaching to the current script/data etc.

sql = 'select * from Counts WHERE count < ? and count > ?'
a = cur.execute(sql,(12,4))

b = a.fetchmany(4)  
print(b)

[('rsmart.com', 8), ('unicon.net', 9), ('loi.nl', 9)]


ASSIGNMENT

In [3]:
import sqlite3
conn = sqlite3.connect('emaildb.sqlite')
cur = conn.cursor()

cur.execute('DROP TABLE IF EXISTS Counts')

cur.execute('CREATE TABLE Counts (org TEXT, count INTEGER)')

fname = input('Enter file name: ')
if (len(fname) < 1): fname = 'mbox.txt'
fh = open(fname)
for line in fh:
    if not line.startswith('From: '): continue
    org = line.split()[1].split('@')[1]

    cur.execute('SELECT count FROM Counts WHERE org = ? ', (org,))
    row = cur.fetchone()
    if row is None:
        cur.execute('''INSERT INTO Counts (org, count)
                VALUES (?, 1)''', (org,))
    else:
        cur.execute('UPDATE Counts SET count = count + 1 WHERE org = ?',
                    (org,))

conn.commit()

# https://www.sqlite.org/lang_select.html
sqlstr = 'SELECT org, count FROM Counts ORDER BY count DESC LIMIT 10'
sqlcnt = 'SELECT count(*) FROM Counts'
for row in cur.execute(sqlstr):
    print(str(row[0]), row[1])
for row in cur.execute(sqlcnt):
    print('total org', row[0])

cur.close()



Enter file name: 
iupui.edu 536
umich.edu 491
indiana.edu 178
caret.cam.ac.uk 157
vt.edu 110
uct.ac.za 96
media.berkeley.edu 56
ufp.pt 28
gmail.com 25
et.gatech.edu 17
total org 21


DATA MODEL DESIGN  

In [ ]:
#this is a helpful piece of code for looking up 
def lookup(d, keyword):
    found = False
    for child in d:
        if found : return child.text
        if child.tag == 'key' and child.text == keyword :
            found = True
    return None


In [5]:
import xml.etree.ElementTree as ET
import sqlite3

conn = sqlite3.connect('trackdb.sqlite')
cur = conn.cursor()

# Make some fresh tables using executescript()
cur.executescript('''
DROP TABLE IF EXISTS Artist;
DROP TABLE IF EXISTS Album;
DROP TABLE IF EXISTS Track;

CREATE TABLE Artist (
    id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    name    TEXT UNIQUE
);

CREATE TABLE Album (
    id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    artist_id  INTEGER,
    title   TEXT UNIQUE
);

CREATE TABLE Track (
    id  INTEGER NOT NULL PRIMARY KEY 
        AUTOINCREMENT UNIQUE,
    title TEXT  UNIQUE,
    album_id  INTEGER,
    len INTEGER, rating INTEGER, count INTEGER
);
''')


fname = input('Enter file name: ')
if ( len(fname) < 1 ) : fname = './code3/tracks/Library.xml'

# <key>Track ID</key><integer>369</integer>
# <key>Name</key><string>Another One Bites The Dust</string>
# <key>Artist</key><string>Queen</string>
def lookup(d, key):
    found = False
    for child in d:
        if found : return child.text
        if child.tag == 'key' and child.text == key :
            found = True
    return None

stuff = ET.parse(fname)
all = stuff.findall('dict/dict/dict')
print('Dict count:', len(all))
for entry in all:
    if ( lookup(entry, 'Track ID') is None ) : continue

    name = lookup(entry, 'Name')
    artist = lookup(entry, 'Artist')
    album = lookup(entry, 'Album')
    count = lookup(entry, 'Play Count')
    rating = lookup(entry, 'Rating')
    length = lookup(entry, 'Total Time')

    if name is None or artist is None or album is None : 
        continue

    print(name, artist, album, count, rating, length)

    cur.execute('''INSERT OR IGNORE INTO Artist (name) 
        VALUES ( ? )''', ( artist, ) )
    cur.execute('SELECT id FROM Artist WHERE name = ? ', (artist, ))
    artist_id = cur.fetchone()[0]

    cur.execute('''INSERT OR IGNORE INTO Album (title, artist_id) 
        VALUES ( ?, ? )''', ( album, artist_id ) )
    cur.execute('SELECT id FROM Album WHERE title = ? ', (album, ))
    album_id = cur.fetchone()[0]

    cur.execute('''INSERT OR REPLACE INTO Track
        (title, album_id, len, rating, count) 
        VALUES ( ?, ?, ?, ?, ? )''', 
        ( name, album_id, length, rating, count ) )

conn.commit()
cur.close()


Enter file name: 
Dict count: 404
Another One Bites The Dust Queen Greatest Hits 55 100 217103
Asche Zu Asche Rammstein Herzeleid 79 100 231810
Beauty School Dropout Various Grease 48 100 239960
Black Dog Led Zeppelin IV 109 100 296620
Bring The Boys Back Home Pink Floyd The Wall [Disc 2] 33 100 87118
Circles Bryan Lee Blues Is 54 60 355369
Comfortably Numb Pink Floyd The Wall [Disc 2] 36 100 384130
Crazy Little Thing Called Love Queen Greatest Hits 38 100 163631
Electric Funeral Black Sabbath Paranoid 44 100 293015
Fat Bottomed Girls Queen Greatest Hits 38 100 257515
For Those About To Rock (We Salute You) AC/DC Who Made Who 84 100 353750
Four Sticks Led Zeppelin IV 84 100 284421
Furious Angels Rob Dougan The Matrix Reloaded 54 100 330004
Gelle Bryan Lee Blues Is 45 60 199836
Going To California Led Zeppelin IV 100 100 215666
Grease Various Grease 42 100 205792
Hand of Doom Black Sabbath Paranoid 36 100 429609
Hells Bells AC/DC Who Made Who 82 100 312946
Hey You Pink Floyd The Wall [D

Hi metal man Brent Brent's Album 4 None 260153
Mistro Brent Brent's Album 5 None 178076
Pirate spirit Brent Brent's Album 3 None 180480
Run Away (New)  Brent Brent's Album 1 None 154383
Star Gaze (Inspired) Charlie And The Rising Moon Charlie and The Rising Moon None None 181498
The Dictator (New Brent Peanut Butter & Jam  5 None 143830
Town From Town Brent Brent's Album 4 None 86073
The Wisdom of Crowds: Why the Many Are Smarter than the Few Part 1 of 3 James Surowiecki The Wisdom of Crowds: Why the Many Are Smarter than the Few (Abridged Nonfiction) 15 None 8586749
The Wisdom of Crowds: Why the Many Are Smarter than the Few Part 2 of 3 James Surowiecki The Wisdom of Crowds: Why the Many Are Smarter than the Few (Abridged Nonfiction) 9 None 8443432
The Wisdom of Crowds: Why the Many Are Smarter than the Few Part 3 of 3 James Surowiecki The Wisdom of Crowds: Why the Many Are Smarter than the Few (Abridged Nonfiction) 3 None 4168829
The Wisdom of Crowds: Why the Many Are Smarter Than th

PHP-12-Sessions.mp3 Created by Sakai SI 664 W14's official Podcast. None None 3624312
PHP-06-Objects.mp3 Created by Sakai SI 664 W14's official Podcast. None None 2455368
SI664-13-JSON-A.mp3 Created by Sakai SI 664 W14's official Podcast. None None 4407672
PHP-13-JavaScript.mp3 Created by Sakai SI 664 W14's official Podcast. None None 3650088
PHP-09-Database-Design.mp3 Created by Sakai SI 664 W14's official Podcast. None None 3591072
PHP-Tsugi-Install.mp3 Created by Sakai SI 664 W14's official Podcast. None None 3181656
PHP-10-MySQL-PDO.mp3 Created by Sakai SI 664 W14's official Podcast. None None 4536685
PHP-08-MySQL.mp3 Created by Sakai SI 664 W14's official Podcast. None None 3994656
SI664-13-JSON-B.mp3 Created by Sakai SI 664 W14's official Podcast. None None 1484616
IMS-Learning-Tools-Interoperability.mp3 Created by Sakai SI 664 W14's official Podcast. None None 2185848
An Interview with Don Waters Matt Pasiewicz CNI Event Coverage 2 None 1411082
PY4INF-11-Regex.mp3 Recording by D

In [8]:
import xml.etree.ElementTree as ET
import sqlite3

fname = input('Enter file name: ')
if ( len(fname) < 1 ) : fname = './code3/tracks/Library.xml'

# <key>Track ID</key><integer>369</integer>
# <key>Name</key><string>Another One Bites The Dust</string>
# <key>Artist</key><string>Queen</string>
def lookup(d, key):
    found = False
    for child in d:
        if found : return child.text
        if child.tag == 'key' and child.text == key :
            found = True
    return None

stuff = ET.parse(fname)
all = stuff.findall('dict/dict/dict')
print('Dict count:', len(all))
for entry in all:
    for child in entry:
        print(child.tag, child.text)



Enter file name: 
Dict count: 404
key Track ID
integer 369
key Name
string Another One Bites The Dust
key Artist
string Queen
key Composer
string John Deacon
key Album
string Greatest Hits
key Genre
string Rock
key Kind
string MPEG audio file
key Size
integer 4344295
key Total Time
integer 217103
key Disc Number
integer 1
key Disc Count
integer 1
key Track Number
integer 3
key Track Count
integer 17
key Year
integer 1980
key Date Modified
date 2006-02-14T16:13:02Z
key Date Added
date 2006-02-14T16:12:53Z
key Bit Rate
integer 160
key Sample Rate
integer 44100
key Play Count
integer 55
key Play Date
integer 3518868190
key Play Date UTC
date 2015-07-04T19:23:10Z
key Skip Count
integer 1
key Skip Date
date 2015-10-14T23:31:47Z
key Rating
integer 100
key Album Rating
integer 100
key Album Rating Computed
true None
key Normalization
integer 1511
key Compilation
true None
key Persistent ID
string 21130E105F3B8845
key Track Type
string File
key File Type
integer 1297106739
key Location
string 

key Date Modified
date 2006-02-14T16:15:38Z
key Date Added
date 2006-02-14T16:15:24Z
key Bit Rate
integer 160
key Sample Rate
integer 44100
key Play Count
integer 84
key Play Date
integer 3503057548
key Play Date UTC
date 2015-01-02T20:32:28Z
key Rating
integer 100
key Album Rating
integer 100
key Album Rating Computed
true None
key Normalization
integer 897
key Persistent ID
string 21130E105F3B8850
key Track Type
string File
key File Type
integer 1297106739
key Location
string file:///Users/csev/Music/iTunes/iTunes%20Music/Led%20Zeppelin/IV/06%20Four%20Sticks.mp3
key File Folder Count
integer 4
key Library Folder Count
integer 1
key Track ID
integer 393
key Name
string Furious Angels
key Artist
string Rob Dougan
key Composer
string Rob Dougan
key Album
string The Matrix Reloaded
key Genre
string Soundtrack
key Kind
string MPEG audio file
key Size
integer 6602318
key Total Time
integer 330004
key Disc Number
integer 1
key Disc Count
integer 2
key Track Number
integer 4
key Track Count


key Bit Rate
integer 160
key Sample Rate
integer 44100
key Play Count
integer 23
key Play Date
integer 3514729106
key Play Date UTC
date 2015-05-17T21:38:26Z
key Rating
integer 100
key Album Rating
integer 100
key Album Rating Computed
true None
key Normalization
integer 1309
key Sort Album
string Wall [Disc 2]
key Persistent ID
string 21130E105F3B8858
key Track Type
string File
key File Type
integer 1297106739
key Location
string file:///Users/csev/Music/iTunes/iTunes%20Music/Pink%20Floyd/The%20Wall%20%5BDisc%202%5D/2-01%20Hey%20You.mp3
key File Folder Count
integer 4
key Library Folder Count
integer 1
key Track ID
integer 409
key Name
string I Worry
key Artist
string Bryan Lee
key Album
string Blues Is
key Genre
string Blues/R&B
key Kind
string MPEG audio file
key Size
integer 6828475
key Total Time
integer 341315
key Track Number
integer 11
key Track Count
integer 12
key Date Modified
date 2006-02-14T16:17:56Z
key Date Added
date 2006-02-14T16:17:39Z
key Bit Rate
integer 160
key Sam

key Sample Rate
integer 44100
key Play Count
integer 41
key Play Date
integer 3483455112
key Play Date UTC
date 2014-05-20T22:25:12Z
key Skip Count
integer 1
key Skip Date
date 2013-09-20T23:58:48Z
key Rating
integer 100
key Album Rating
integer 100
key Album Rating Computed
true None
key Normalization
integer 3721
key Persistent ID
string 21130E105F3B8860
key Track Type
string File
key File Type
integer 1297106739
key Location
string file:///Users/csev/Music/iTunes/iTunes%20Music/Rammstein/Herzeleid/10%20Laichzeit.mp3
key File Folder Count
integer 4
key Library Folder Count
integer 1
key Track ID
integer 425
key Name
string Let me Down Easy
key Artist
string Bryan Lee
key Album
string Blues Is
key Genre
string Blues/R&B
key Kind
string MPEG audio file
key Size
integer 6630998
key Total Time
integer 331441
key Track Number
integer 4
key Track Count
integer 12
key Date Modified
date 2006-02-14T16:19:50Z
key Date Added
date 2006-02-14T16:19:36Z
key Bit Rate
integer 160
key Sample Rate
in

key Normalization
integer 4946
key Persistent ID
string 21130E105F3B886B
key Track Type
string File
key File Type
integer 1297106739
key Location
string file:///Users/csev/Music/iTunes/iTunes%20Music/Bryan%20Lee/Blues%20Is/10%20Pretty%20Jeanie.mp3
key File Folder Count
integer 4
key Library Folder Count
integer 1
key Track ID
integer 441
key Name
string Rammstein
key Artist
string Rammstein
key Composer
string Christoph Doom Schneider, Doktor Christian Lorenz, Oliver Riedel, Paul Landers, Richard Z. Kruspe-Bernstein & Till Lindemann
key Album
string Herzeleid
key Genre
string Industrial
key Kind
string MPEG audio file
key Size
integer 5304122
key Total Time
integer 265090
key Disc Number
integer 1
key Disc Count
integer 1
key Track Number
integer 11
key Track Count
integer 11
key Year
integer 1995
key Date Modified
date 2011-11-18T19:28:29Z
key Date Added
date 2006-02-14T16:21:34Z
key Bit Rate
integer 160
key Sample Rate
integer 44100
key Play Count
integer 45
key Play Date
integer 348

...

key Genre
string Podcast
key Kind
string MPEG audio file
key Size
integer 20631428
key Total Time
integer 2578416
key Year
integer 2013
key BPM
integer 120
key Date Modified
date 2014-11-15T14:18:17Z
key Date Added
date 2014-11-15T14:18:17Z
key Bit Rate
integer 64
key Sample Rate
integer 44100
key Release Date
date 2013-09-18T01:03:00Z
key Normalization
integer 1245
key Persistent ID
string 7CE111FA768BF10F
key Track Type
string File
key Podcast
true None
key Location
string file:///Users/csev/Music/iTunes/iTunes%20Music/Podcasts/Python%20for%20Informatics's%20official%20Podcast_/PY4INF-02-Expressions.mp3%201.mp3
key File Folder Count
integer 4
key Library Folder Count
integer 1
key Track ID
integer 1141
key Name
string PY4INF-01-Intro.mp3
key Artist
string Recording by Dr. Chuck
key Composer
string www.dr-chuck.com
key Album
string Python for Informatics's official Podcast.
key Genre
string Podcast
key Kind
string MPEG audio file
key Size
integer 27428698
key Total Time
integer 342807

key Location
string file:///Users/csev/Music/iTunes/iTunes%20Music/Podcasts/Python%20for%20Informatics's%20official%20Podcast_/Py4Inf-01-Intro.mp3%202.mp3
key File Folder Count
integer 4
key Library Folder Count
integer 1
key Track ID
integer 1175
key Name
string Python 1994 Workshop 2-HD (1080p)
key Kind
string MPEG-4 video file
key Size
integer 26790760
key Total Time
integer 40033
key Date Modified
date 2015-01-20T14:24:15Z
key Date Added
date 2015-01-20T14:24:15Z
key Bit Rate
integer 105
key Artwork Count
integer 1
key Persistent ID
string 5549223E93B9DB8A
key Track Type
string File
key Has Video
true None
key HD
false None
key Video Width
integer 1920
key Video Height
integer 1080
key File Type
integer 1295275552
key Location
string file:///Users/csev/Desktop/NoBackup/Python%201994%20Workshop%202-HD%20(1080p).m4v
key File Folder Count
integer 3
key Library Folder Count
integer 3
key Track ID
integer 1177
key Name
string Python 1994 Workshop 2-HD (1080p)
key Kind
string MPEG-4 vide

string File
key Location
string file:///Applications/MAMP/htdocs/net-intro/book/audio/05-dns.mp4
key File Folder Count
integer 7
key Library Folder Count
integer 5
key Track ID
integer 1213
key Name
string 06-transport-layer
key Kind
string MPEG-4 video file
key Size
integer 2747826
key Total Time
integer 652004
key Date Modified
date 2015-05-21T22:32:31Z
key Date Added
date 2015-06-21T16:37:05Z
key Bit Rate
integer 31
key Persistent ID
string F3D95B4DBCF6B3F5
key Track Type
string File
key Location
string file:///Applications/MAMP/htdocs/net-intro/book/audio/06-transport-layer.mp4
key File Folder Count
integer 7
key Library Folder Count
integer 5
key Track ID
integer 1215
key Name
string 07-application-layer
key Kind
string MPEG-4 video file
key Size
integer 4897176
key Total Time
integer 1168282
key Date Modified
date 2015-05-21T22:32:38Z
key Date Added
date 2015-06-21T16:37:05Z
key Bit Rate
integer 31
key Persistent ID
string EA268C5731134645
key Track Type
string File
key Location


key Name
string Wounded Bird
key Artist
string The Black Crowes
key Album Artist
string The Black Crowes
key Composer
string Chris Robinson & Rich Robinson
key Album
string Warpaint
key Genre
string Rock
key Kind
string Purchased AAC audio file
key Size
integer 4320806
key Total Time
integer 263453
key Disc Number
integer 1
key Disc Count
integer 1
key Track Number
integer 8
key Track Count
integer 12
key Year
integer 2008
key Date Modified
date 2015-01-06T18:11:06Z
key Date Added
date 2008-03-10T03:44:26Z
key Bit Rate
integer 128
key Sample Rate
integer 44100
key Release Date
date 2008-02-29T08:00:00Z
key Artwork Count
integer 1
key Sort Album
string Warpaint
key Sort Album Artist
string Black Crowes
key Sort Artist
string Black Crowes
key Sort Name
string Wounded Bird
key Persistent ID
string 383A5BCF1861134B
key Track Type
string Remote
key Purchased
true None
key Track ID
integer 1243
key Name
string Volcano
key Artist
string U2
key Album Artist
string U2
key Album
string Songs of 

string Undercover Boss
key Sort Name
string Choice Hotels International
key Persistent ID
string 58277AAEDC4D207B
key Content Rating
string us-tv|TV-PG|400|
key Track Type
string Remote
key Purchased
true None
key Has Video
true None
key HD
false None
key TV Show
true None
key Track ID
integer 1285
key Name
string Microchip
key Artist
string Jason Farnham
key Album
string YouTube Audio Library
key Genre
string Dance & Electronic
key Kind
string MPEG audio file
key Size
integer 4573582
key Total Time
integer 114336
key Date Modified
date 2015-09-16T02:14:00Z
key Date Added
date 2015-09-16T02:16:35Z
key Bit Rate
integer 320
key Sample Rate
integer 48000
key Persistent ID
string 15BA4C571757AB93
key Track Type
string File
key Location
string file:///Users/csev/Downloads/Microchip.mp3
key File Folder Count
integer 2
key Library Folder Count
integer 3
key Track ID
integer 1287
key Name
string Cookies-01
key Kind
string MPEG audio file
key Size
integer 44917520
key Total Time
integer 2807248

ASSIGNMENT